In [1]:
import pandas as pd
import numpy as np
import os

#### Importation des données pour variable auxiliaire

In [13]:
chomage=pd.read_excel("data/taux_chomage.xlsx")
richesse=pd.read_excel("data/richesse.xlsx")
inondation=pd.read_excel("data/Gaspar.xlsx")

In [14]:
display(chomage)
#richesse.drop(columns="Unnamed: 0",inplace=True)
display(richesse)

display(inondation)

,Code,MOYENNE
0,01,8.700
1,02,6.970
2,03,13.370
3,04,10.060
4,05,10.960
...,...,...
92,92,7.420
93,93,7.410
94,94,12.485
95,95,8.620


,Unnamed: 0,Loyer moyen par mètre carré de surface habitable (en €),Taux de logements sociaux
0,01,5.9000,7.1600
1,02,5.3300,7.7900
2,03,5.2200,6.0900
3,04,5.8700,4.5500
4,05,5.3200,5.6900
...,...,...,...
92,92,7.1700,12.4700
93,93,6.7800,14.0800
94,94,6.9400,12.6500
95,95,6.5800,10.1200


,Commune,Somme de nb_com_ddrm
0,01001,1
1,01002,0
2,01004,1
3,01005,0
4,01006,0
...,...,...
34959,97612,1
34960,97613,1
34961,97614,1
34962,97615,1


In [20]:
richesse.set_index("Unnamed: 0",inplace=True)
chomage.set_index("Code",inplace=True)
inondation.set_index("Commune",inplace=True)

In [21]:
chomage.set_index("Code",inplace=True)

In [25]:
inondation.set_index("Commune",inplace=True)

On a deux types de jeu de données: 
- Un basé sur les codes postaux des communes
- L'autre sur les départements

On dispose des deux colonnes dans le dataframe de base, ce qui va nous permettre de réaliser facilement un merge:
- DEPARTEMENT_CRI

#### Analyse exploratoire des proxys

In [26]:
#On établi les 3 dictionnaires pour les variables socio-démographique

def replace_if_not_in(x,list_geo,col):
    if x not in list_geo:
        return col.value_counts().idxmax()
    else: 
        return x
def proxys_processing(df,richesse,chomage,inondation):
    richesse.set_index("Unnamed: 0",inplace=True)
    chomage.set_index("Code",inplace=True)
    inondation.set_index("Commune",inplace=True)
    dictionary_chomage=chomage.to_dict()['MOYENNE']
    dictionary_hlm=richesse.to_dict()['Taux de logements sociaux']
    dictionary_prix_loyer=richesse.to_dict()['Loyer moyen par mètre carré de surface habitable (en €)']
    dictionary_innondation=inondation.to_dict()['Somme de nb_com_ddrm']
    departement_proxy=chomage.index.astype(str).tolist()
    codes_postaux_proxy=inondation.index.astype(str).tolist()
    df['DEPARTEMENT_CRI']=df['DEPARTEMENT_CRI'].astype(str).apply(lambda x: replace_if_not_in(x,departement_proxy,df["DEPARTEMENT_CRI"]))
    df["TAUX_HLM"]=df["DEPARTEMENT_CRI"].astype(str).map(dictionary_hlm)
    df["TAUX_CHOMAGE"]=df["DEPARTEMENT_CRI"].astype(str).map(dictionary_chomage)
    df["PRIX_LOYER"]=df["DEPARTEMENT_CRI"].map(dictionary_prix_loyer)
    df["TAUX_HLM"]=df["TAUX_HLM"].fillna(df["TAUX_HLM"].median())
    df["TAUX_CHOMAGE"]=df["TAUX_CHOMAGE"].fillna(df["TAUX_CHOMAGE"].median())
    df["PRIX_LOYER"]=df["PRIX_LOYER"].fillna(df["PRIX_LOYER"].median())


In [61]:

departement_proxy=chomage.index.astype(str).tolist()
codes_postaux_proxy=inondation.index.astype(str).tolist()

def replace_if_not_in(x,list_geo,col):
    if x not in list_geo:
        return col.value_counts().idxmax()
    else: 
        return x

df['DEPARTEMENT_CRI']=df['DEPARTEMENT_CRI'].astype(str).apply(lambda x: replace_if_not_in(x,departement_proxy,df["DEPARTEMENT_CRI"]))
df["TAUX_HLM"]=df["DEPARTEMENT_CRI"].astype(str).map(dictionary_hlm)
df["TAUX_CHOMAGE"]=df["DEPARTEMENT_CRI"].astype(str).map(dictionary_chomage)
df["PRIX_LOYER"]=df["DEPARTEMENT_CRI"].map(dictionary_prix_loyer)
df["TAUX_HLM"]=df["TAUX_HLM"].fillna(df["TAUX_HLM"].median())
df["TAUX_CHOMAGE"]=df["TAUX_CHOMAGE"].fillna(df["TAUX_CHOMAGE"].median())
df["PRIX_LOYER"]=df["PRIX_LOYER"].fillna(df["PRIX_LOYER"].median())